# 한국어 텍스트 감정 분석 - PyTorch 모델 학습

이 노트북은 전처리된 데이터를 사용하여 PyTorch 기반의 BERT/RoBERTa 모델을 학습합니다.

## 주요 기능
- PyTorch 네이티브 학습 루프 (Transformers Trainer 대신)
- LoRA 파인튜닝
- WandB 실험 추적
- 혼합 정밀도 학습 (Mixed Precision)
- 조기 종료 및 모델 체크포인팅
- 테스트 데이터 추론 및 제출 파일 생성


In [1]:
# Library Import
import os
import math
import warnings
from collections import Counter
import platform
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    set_seed
)

# PEFT for LoRA
from peft import get_peft_model, LoraConfig, TaskType

# Sklearn
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

print("✅ 라이브러리 임포트 완료")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

✅ 라이브러리 임포트 완료


In [2]:
# 환경 설정
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()

RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")



디바이스: cuda
GPU 개수: 1
   GPU 0: Tesla V100-SXM2-32GB


In [3]:
import datetime
now = datetime.datetime.now()
TIMESTAMP = now.strftime("%Y-%m-%d_%H-%M-%S")

In [4]:
molels = ["beomi_kcbert-base", "klue_roberta-base", "klue_bert-base","kykim_bert-base", "monologg_koelectra-base-v3-discriminator"]

# 라벨 매핑
LABEL_MAPPING = {0: "강한 부정", 1: "약한 부정", 2: "약한 긍정", 3: "강한 긍정"}

In [5]:
RUN_NAME = f"newly_gen_added"

PROJECT_NAME = f"[domain_project]_Experiments"
MODEL_NAME = "kykim_bert-base"

os.environ["TOKENIZERS_PARALLELISM"] = "false"




In [6]:
# 하이퍼파라미터 설정

class Config:
    # 모델 설정

    model_name = MODEL_NAME
    base_models_path = "/data/ephemeral/home/code/basemodels/"
    local_model_path = base_models_path + MODEL_NAME

    is_DAPT = True
    #DAPT_model_path = "./DAPT_models/"+ "kykim_bert-base_unsupervised_model_1025"
    DAPT_model_path = "./kykim_bert-kor-base_augX2_TAPT_best_unsupervised_model_1026"
    
    save_model_path = "./DAPT_fine/"+ RUN_NAME

    num_classes = 4
    max_length = 128

    # 데이터 설정
    train_data_path = "/data/ephemeral/home/code/data/train_processed_newly_gen_added.csv"
    val_data_path = "/data/ephemeral/home/code/data/val_processed_newly_gen_added.csv"
    
    # 훈련 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    batch_size = 512
    eval_batch_size = 512
    num_epochs = 5
    learning_rate = 2e-5
    weight_decay = 0.01
    warmup_steps = 500
    
    # 기타 설정
    gradient_accumulation_steps = 1
    max_grad_norm = 1.0
    early_stopping_patience = 5
    save_best_model = True
    
    # WandB 설정
    use_wandb = True
    project_name = PROJECT_NAME
    run_name = f"{RUN_NAME}-training"

    # Loss 설정
    loss_function = "CrossEntropy"
    use_label_smoothing = True
    label_smoothing_factor = 0.1  # 추가 필요

    use_lora = False
    lora_r = 8                    # LoRA rank (차원)
    lora_alpha = 16                # LoRA alpha (스케일링)
    lora_dropout = 0.1             # LoRA dropout
    lora_target_modules = ["query", "key", "value", "dense"]  # LoRA 적용할 모듈들
    lora_bias = "none"             # bias 설정: "none", "all", "lora_only"
    lora_task_type = "SEQ_CLS"     # 태스크 타입



config = Config()
print("✅ 하이퍼파라미터 설정 완료")


✅ 하이퍼파라미터 설정 완료


In [7]:
# WandB 초기화
if config.use_wandb:
    run = wandb.init(
        project=config.project_name,
        name=config.run_name,
        config={
            "model_name": config.model_name,
            "num_epochs": config.num_epochs,
            "batch_size": config.batch_size,
            "learning_rate": config.learning_rate,
            "max_length": config.max_length,
            "num_classes": config.num_classes,
            "warmup_steps": config.warmup_steps,
            "weight_decay": config.weight_decay,
            "lora_r": config.lora_r,
            "lora_alpha": config.lora_alpha,
            "lora_dropout": config.lora_dropout,
            "random_seed": RANDOM_STATE
        }
    )
    print("✅ WandB 초기화 완료")
else:
    print("⚠️  WandB 사용 안함")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


✅ WandB 초기화 완료


In [8]:
# 데이터 로드
print("전처리된 데이터 로드 중...")

# 훈련 데이터 로드
train_df = pd.read_csv(config.train_data_path)
print(f"훈련 데이터: {len(train_df):,}개")

# 검증 데이터 로드
val_df = pd.read_csv(config.val_data_path)
print(f"검증 데이터: {len(val_df):,}개")


print("✅ 데이터 로드 완료")


전처리된 데이터 로드 중...
훈련 데이터: 330,248개
검증 데이터: 36,695개
✅ 데이터 로드 완료


In [ ]:
dataset_artifect = wandb.Artifact(
    name="preprocessed_dataset_augX2",
    type="dataset",
    description="split the augmented data 데이터 셋"
)

dataset_artifect.add_file("data/train_processed.csv")
dataset_artifect.add_file("data/val_processed.csv")

run.log_artifact(dataset_artifect)



In [9]:
# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    """
    리뷰 텍스트 데이셋 클래스
    - BERT 모델 훈련/추론을 위한 PyTorch Dataset 구현
    - 텍스트 토크나이징 및 텐서 변환 처리
    """

    def __init__(self, texts, labels, tokenizer, max_length):
        """
        데이터셋 초기화
        """
        self.texts, self.labels, self.tokenizer, self.max_length = (
            texts,
            labels,
            tokenizer,
            max_length,
        )

    def __len__(self):
        """데이터셋 크기 반환"""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        특정 인덱스의 데이터 아이템 반환
        """
        # 텍스트 토크나이징 및 패딩
        encoding = self.tokenizer(
            str(self.texts.iloc[idx]),
            truncation=True,  # 최대 길이 초과시 자르기
            padding="max_length",  # 최대 길이까지 패딩
            max_length=self.max_length,
            return_tensors="pt",  # PyTorch 텐서로 반환
        )

        # 기본 아이템 구성 (input_ids, attention_mask)
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

        # labels가 None이 아닌 경우에만 추가 (train/valid용)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)

        return item

print("✅ 데이터셋 클래스 정의 완료")


✅ 데이터셋 클래스 정의 완료


In [10]:
# 평가 메트릭 함수
def compute_metrics(predictions, labels):
    """
    모델 평가 메트릭 계산 함수
    """
    # 예측값에서 가장 높은 확률의 클래스 선택
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
    }

print("✅ 평가 메트릭 함수 정의 완료")


✅ 평가 메트릭 함수 정의 완료


In [11]:
def model_load_from_local(local_model_path : str = None, model_name : str = None):
    # 1. 토크나이저 로드
    # 절대 경로를 사용하여 로컬 모델 로드
    print("로컬 경로에서 토크나이저 로딩 중...")

    # 경로가 존재하는지 확인
    if not os.path.exists(local_model_path):
        print(f"❌ 경로가 존재하지 않습니다: {local_model_path}")
        return None, None
    else:
        print(f"✅ 경로 확인됨: {local_model_path}")

    tokenizer = AutoTokenizer.from_pretrained(local_model_path)

    # 2. 모델 로드
    # 마찬가지로 로컬 경로(local_model_path)를 사용
    print("로컬 경로에서 모델 로딩 중...")
    model = AutoModelForSequenceClassification.from_pretrained(
        local_model_path,
        num_labels=4,
        ignore_mismatched_sizes=True
    )

    print("✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!")

    # --- 이제 평소처럼 모델을 사용할 수 있습니다 ---
    inputs = tokenizer("이 영화 정말 재미있네요!", return_tensors="pt")
    outputs = model(**inputs)
    print(outputs.logits)

    return model, tokenizer

In [12]:
if config.is_DAPT:
    model, tokenizer = model_load_from_local(config.DAPT_model_path, config.model_name)
else:
    model, tokenizer = model_load_from_local(config.local_model_path, config.model_name)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./kykim_bert-kor-base_augX2_TAPT_best_unsupervised_model_1026 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


로컬 경로에서 토크나이저 로딩 중...
✅ 경로 확인됨: ./kykim_bert-kor-base_augX2_TAPT_best_unsupervised_model_1026
로컬 경로에서 모델 로딩 중...
✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!
tensor([[-0.0761,  0.2847, -0.2230,  0.0559]], grad_fn=<AddmmBackward0>)


In [13]:
#LoRA 설정 적용
# print("LoRA 설정 적용 중...")
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,
#     r=config.lora_r,
#     lora_alpha=config.lora_alpha,
#     target_modules=["query", "key", "value"],
#     lora_dropout=config.lora_dropout,
#     bias="none",
# )

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

# 모델을 디바이스로 이동
model = model.to(device)

print("✅ 모델 및 토크나이저 로드 완료")


✅ 모델 및 토크나이저 로드 완료


### LOSS 

In [14]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class FocalLoss(nn.Module):
#     """
#     Focal Loss 구현 (개선된 버전)
#     - 클래스 불균형 문제 해결을 위한 손실 함수
#     - 쉬운 샘플의 기여도를 줄이고 어려운 샘플에 집중
#     """
#     def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
#         """
#         Args:
#             alpha (list, tensor, float, int, optional): 
#                 - list/tensor: 클래스별 가중치. [num_classes] 형태.
#                 - float/int: 모든 클래스에 동일하게 적용될 단일 가중치.
#             gamma (float): focusing parameter (기본값: 2.0)
#             reduction (str): 'mean', 'sum', 'none'
#         """
#         super(FocalLoss, self).__init__()
        
#         # --- [수정된 부분 1] ---
#         # alpha를 tensor로 변환하고 버퍼로 등록
#         if alpha is not None:
#             if isinstance(alpha, (float, int)):
#                 # 단일 값인 경우, 텐서로 만들 필요 없이 그대로 사용
#                 self.alpha = torch.tensor(alpha, dtype=torch.float32)
#             elif isinstance(alpha, (list, torch.Tensor)):
#                 if isinstance(alpha, list):
#                     alpha_tensor = torch.tensor(alpha, dtype=torch.float32)
#                 else:
#                     alpha_tensor = alpha.clone().detach().to(dtype=torch.float32)
#                 # register_buffer: 모델의 state_dict에 포함되지만, 
#                 # optimzer에 의해 업데이트(학습)되지는 않음.
#                 # .to(device) 호출 시 함께 이동함.
#                 self.register_buffer('alpha', alpha_tensor)
#             else:
#                 raise TypeError("alpha must be a float, int, list, or torch.Tensor")
#         else:
#             self.alpha = None
#         # --- [수정 끝] ---
            
#         self.gamma = gamma
#         self.reduction = reduction

#     def forward(self, inputs, targets):
#         """
#         Args:
#             inputs: 모델 출력 (logits) [batch_size, num_classes]
#             targets: 정답 라벨 [batch_size]
#         """
#         # Cross Entropy Loss 계산 (reduction='none'이 중요)
#         # ce_loss = -log(pt)
#         ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
#         # 확률 계산 (pt = exp(-ce_loss))
#         pt = torch.exp(-ce_loss)
        
#         # Focal Loss 계산
#         focal_loss = (1 - pt) ** self.gamma * ce_loss
        
#         # Alpha 가중치 적용
#         if self.alpha is not None:
#             # --- [수정된 부분 2] ---
#             # self.alpha가 스칼라 텐서인지(float/int에서 변환됨) 
#             # 아니면 벡터 텐서(list에서 변환됨)인지 확인
            
#             # .to(device)를 명시적으로 호출하여 디바이스 일치 보장
#             alpha = self.alpha.to(targets.device) 

#             if alpha.numel() == 1:
#                 # alpha가 단일 값(float/int)에서 온 경우
#                 alpha_t = alpha
#             else:
#                 # alpha가 list/tensor에서 온 경우 (클래스별 가중치)
#                 # targets를 인덱스로 사용
#                 alpha_t = alpha[targets]
#             # --- [수정 끝] ---
                
#             focal_loss = alpha_t * focal_loss
        
#         # Reduction 적용
#         if self.reduction == 'mean':
#             return focal_loss.mean()
#         elif self.reduction == 'sum':
#             return focal_loss.sum()
#         else:
#             return focal_loss

In [15]:
def calculate_class_weights(labels):
    """클래스별 가중치 계산"""
    from sklearn.utils.class_weight import compute_class_weight
    
    classes = np.unique(labels)
    class_weights = compute_class_weight(
        'balanced',
        classes=classes,
        y=labels
    )
    return torch.FloatTensor(class_weights)

train_labels = train_df["label"].values
class_counts = np.bincount(train_labels)
print("클래스별 분포:")
for i, count in enumerate(class_counts):
    print(f"  클래스 {i}: {count:,}개 ({count/len(train_labels)*100:.1f}%)")

# Focal Loss용 Alpha 계산
# if config.use_focal_loss:
#     if config.focal_alpha is None:
#         # 자동으로 클래스 가중치 계산
#         focal_alpha = calculate_class_weights(train_labels)
#         print(f"계산된 Alpha 가중치: {focal_alpha}")
#     else:
#         focal_alpha = torch.FloatTensor(config.focal_alpha)
#         print(f"설정된 Alpha 가중치: {focal_alpha}")

클래스별 분포:
  클래스 0: 121,917개 (36.9%)
  클래스 1: 44,751개 (13.6%)
  클래스 2: 108,355개 (32.8%)
  클래스 3: 55,225개 (16.7%)


In [16]:
import torch.nn.functional as F

def label_smoothing_cross_entropy(predictions, targets, smoothing=0.1):
    """
    predictions: 모델 출력 (logits) [batch_size, num_classes]
    targets: 정답 라벨 (정수) [batch_size]
    smoothing: smoothing factor
    """
    num_classes = predictions.size(-1)
    
    # 하드 라벨을 원-핫으로 변환
    true_dist = torch.zeros_like(predictions)
    true_dist.fill_(smoothing / (num_classes - 1))
    true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - smoothing)
    
    # KL divergence = Cross Entropy with smoothed labels
    return F.kl_div(F.log_softmax(predictions, dim=1), true_dist, reduction='batchmean')

### LOSS 설정

In [17]:
# 손실 함수 설정
if config.loss_function == "FocalLoss":
    criterion = FocalLoss(
        alpha=focal_alpha,
        gamma=config.focal_gamma,
        reduction=config.focal_reduction
    )
    print("✅ Focal Loss 사용")
elif config.loss_function == "WeightedCrossEntropy":
    weights = calculate_class_weights(train_labels)
    criterion = nn.CrossEntropyLoss(weight=weights.to(device))
    print("✅ Weighted Cross Entropy Loss 사용")
else:
    criterion = nn.CrossEntropyLoss()
    print("✅ Cross Entropy Loss 사용")





✅ Cross Entropy Loss 사용


In [18]:
if config.use_label_smoothing:
    def criterion_fn(predictions, targets):
        return label_smoothing_cross_entropy(predictions, targets, config.label_smoothing_factor)
    print("✅ Label Smoothing Cross Entropy 사용")
else:
    criterion = nn.CrossEntropyLoss()
    print("✅ 일반 Cross Entropy Loss 사용")

✅ Label Smoothing Cross Entropy 사용


## 학습 준비

In [19]:
# 데이터셋 및 데이터로더 생성
print("데이터셋 및 데이터로더 생성 중...")

# 훈련 데이터셋 생성
train_dataset = ReviewDataset(
    train_df["review"],
    train_df["label"],
    tokenizer,
    config.max_length,
)

# 검증 데이터셋 생성
val_dataset = ReviewDataset(
    val_df["review"],
    val_df["label"],
    tokenizer,
    config.max_length,
)

# 데이터로더 생성
train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=config.eval_batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"훈련 데이터: {len(train_dataset):,}개")
print(f"검증 데이터: {len(val_dataset):,}개")
print("✅ 데이터셋 및 데이터로더 생성 완료")


데이터셋 및 데이터로더 생성 중...
훈련 데이터: 330,248개
검증 데이터: 36,695개
✅ 데이터셋 및 데이터로더 생성 완료


In [20]:
# 옵티마이저 및 스케줄러 설정
print("옵티마이저 및 스케줄러 설정 중...")

# 옵티마이저 설정
optimizer = AdamW(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay
)

# 전체 훈련 스텝 계산
total_steps = len(train_dataloader) * config.num_epochs // config.gradient_accumulation_steps

# 스케줄러 설정 (warmup + linear decay)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config.warmup_steps,
    num_training_steps=total_steps
)

# 혼합 정밀도 스케일러
scaler = GradScaler() if torch.cuda.is_available() else None

print(f"총 훈련 스텝: {total_steps:,}")
print(f"워밍업 스텝: {config.warmup_steps:,}")
print("✅ 옵티마이저 및 스케줄러 설정 완료")


옵티마이저 및 스케줄러 설정 중...
총 훈련 스텝: 3,230
워밍업 스텝: 500
✅ 옵티마이저 및 스케줄러 설정 완료


In [21]:
def train_epoch(model, dataloader, optimizer, scheduler, scaler, device, config, criterion):

    model.train()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for step, batch in enumerate(progress_bar):
        # 배치를 디바이스로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        # 혼합 정밀도 훈련
        if scaler is not None:
            with autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    # labels=batch_labels  # 제거
                )

                logits = outputs.logits
   
                if config.use_label_smoothing:
                    loss = criterion_fn(logits, batch_labels) / config.gradient_accumulation_steps
                else:
                    loss = criterion(logits, batch_labels)  / config.gradient_accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (step + 1) % config.gradient_accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
        else:
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
                # labels=batch_labels  # 제거
            )
            logits = outputs.logits

            if config.use_label_smoothing:
                loss = criterion_fn(logits, batch_labels) / config.gradient_accumulation_steps
            else:
                loss = criterion(logits, batch_labels)  / config.gradient_accumulation_steps
            
            loss.backward()
            
            if (step + 1) % config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        
        total_loss += loss.item() * config.gradient_accumulation_steps
        
        # 예측값 저장
        predictions.extend(logits.detach().cpu().numpy())
        labels.extend(batch_labels.detach().cpu().numpy())
        
        # 진행률 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item() * config.gradient_accumulation_steps:.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

In [22]:
# 검증 함수
def validate_epoch(model, dataloader, device, criterion):
    """한 에포크 검증 (Focal Loss 지원)"""
    model.eval()
    total_loss = 0
    predictions = []
    labels = []
    
    progress_bar = tqdm(dataloader, desc="Validation")
    
    with torch.no_grad():
        for batch in progress_bar:
            # 배치를 디바이스로 이동
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            batch_labels = batch["labels"].to(device)
            
            # 순전파
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
                # labels=batch_labels  # 제거
            )
            
            logits = outputs.logits

            if config.use_label_smoothing:
                loss = criterion_fn(logits, batch_labels)
            else:
                loss = criterion(logits, batch_labels)
            
            
            total_loss += loss.item()
            
            # 예측값 저장
            predictions.extend(logits.detach().cpu().numpy())
            labels.extend(batch_labels.detach().cpu().numpy())
            
            # 진행률 업데이트
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # 메트릭 계산
    metrics = compute_metrics(predictions, labels)
    metrics['loss'] = total_loss / len(dataloader)
    
    return metrics

print("✅ 검증 함수 정의 완료")

✅ 검증 함수 정의 완료


In [23]:
# 메인 훈련 루프
print("=" * 50)
print("모델 훈련 시작")
print("=" * 50)

# 훈련 정보 출력
print(f"훈련 샘플: {len(train_dataset):,}개")
print(f"검증 샘플: {len(val_dataset):,}개")
print(f"훈련 에포크: {config.num_epochs}회")
print(f"배치 크기: {config.batch_size} (훈련) / {config.eval_batch_size} (검증)")
print(f"학습률: {config.learning_rate}")
print(f"시드값: {RANDOM_STATE}")
print(f"디바이스: {device}")
print(f"WandB 사용: {config.use_wandb}")

# 조기 종료 및 체크포인팅 설정
best_val_accuracy = 0
patience_counter = 0
training_history = []

# 훈련 시작
try:
    for epoch in range(config.num_epochs):
        print(f"\n{'='*20} Epoch {epoch+1}/{config.num_epochs} {'='*20}")
        
        # 훈련
        train_metrics = train_epoch(
            model, train_dataloader, optimizer, scheduler, scaler, device, config, criterion
        )
        
        # 검증
        val_metrics = validate_epoch(model, val_dataloader, device, criterion)
        
        # 결과 출력
        print(f"\n훈련 결과:")
        print(f"  Loss: {train_metrics['loss']:.4f}")
        print(f"  Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"  F1: {train_metrics['f1']:.4f}")
        
        print(f"\n검증 결과:")
        print(f"  Loss: {val_metrics['loss']:.4f}")
        print(f"  Accuracy: {val_metrics['accuracy']:.4f}")
        print(f"  F1: {val_metrics['f1']:.4f}")
        
        # WandB 로깅
        if config.use_wandb:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_metrics['loss'],
                "train_accuracy": train_metrics['accuracy'],
                "train_f1": train_metrics['f1'],
                "val_loss": val_metrics['loss'],
                "val_accuracy": val_metrics['accuracy'],
                "val_f1": val_metrics['f1'],
                "learning_rate": scheduler.get_last_lr()[0]
            })
        
        # 히스토리 저장
        training_history.append({
            'epoch': epoch + 1,
            'train_loss': train_metrics['loss'],
            'train_accuracy': train_metrics['accuracy'],
            'train_f1': train_metrics['f1'],
            'val_loss': val_metrics['loss'],
            'val_accuracy': val_metrics['accuracy'],
            'val_f1': val_metrics['f1']
        })
        
        # 최고 성능 모델 저장
        if config.save_best_model and val_metrics['accuracy'] > best_val_accuracy:
            best_val_accuracy = val_metrics['accuracy']
            patience_counter = 0
            
            # 모델 저장
            os.makedirs(config.save_model_path, exist_ok=True)
            model.save_pretrained(f"{config.save_model_path}_{TIMESTAMP}")
            tokenizer.save_pretrained(f"{config.save_model_path}_{TIMESTAMP}")

            print(f"✅ 최고 성능 모델 저장 (Accuracy: {best_val_accuracy:.4f})")
        else:
            patience_counter += 1
        
        # 조기 종료 체크
        if patience_counter >= config.early_stopping_patience:
            print(f"\n⚠️  조기 종료: {config.early_stopping_patience} 에포크 동안 개선 없음")
            break
    
    print(f"\n✅ 훈련 완료!")
    print(f"최고 검증 정확도: {best_val_accuracy:.4f}")
    
except KeyboardInterrupt:
    print("\n⚠️  사용자에 의해 훈련이 중단되었습니다.")
except Exception as e:
    print(f"\n❌ 훈련 중 오류 발생: {str(e)}")
    raise


모델 훈련 시작
훈련 샘플: 330,248개
검증 샘플: 36,695개
훈련 에포크: 5회
배치 크기: 512 (훈련) / 512 (검증)
학습률: 2e-05
시드값: 42
디바이스: cuda
WandB 사용: True

==================== Epoch 1/5 ====================


Training:   0%|          | 0/646 [00:00<?, ?it/s]

Validation:   0%|          | 0/72 [00:00<?, ?it/s]


훈련 결과:
  Loss: 0.3507
  Accuracy: 0.7816
  F1: 0.7793

검증 결과:
  Loss: 0.2294
  Accuracy: 0.8659
  F1: 0.8651
✅ 최고 성능 모델 저장 (Accuracy: 0.8659)

==================== Epoch 2/5 ====================


Training:   0%|          | 0/646 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/72 [00:00<?, ?it/s]


훈련 결과:
  Loss: 0.1573
  Accuracy: 0.9109
  F1: 0.9104

검증 결과:
  Loss: 0.2138
  Accuracy: 0.8795
  F1: 0.8796
✅ 최고 성능 모델 저장 (Accuracy: 0.8795)

==================== Epoch 5/5 ====================


Training:   0%|          | 0/646 [00:00<?, ?it/s]

Validation:   0%|          | 0/72 [00:00<?, ?it/s]


훈련 결과:
  Loss: 0.1415
  Accuracy: 0.9212
  F1: 0.9209

검증 결과:
  Loss: 0.2166
  Accuracy: 0.8806
  F1: 0.8804
✅ 최고 성능 모델 저장 (Accuracy: 0.8806)

✅ 훈련 완료!
최고 검증 정확도: 0.8806


In [ ]:
model_artifect = wandb.Artifact(
    name="model",
    type="model",
    description="config.model_name"
)
model_artifect.add_dir(f"{config.save_model_path}_{TIMESTAMP}")
run.log_artifact(model_artifect)

wandb: Adding directory to artifact (DAPT_fine/newly_gen_added_2025-10-28_01-51-51)... Done. 3.0s


<Artifact model>

In [27]:
# 테스트 데이터 추론
print("테스트 데이터 추론 시작...")

# 테스트 데이터 로드
test_df = pd.read_csv("data/test_processed.csv")
print(f"테스트 데이터: {len(test_df):,}개")

# 최고 성능 모델 로드
#if os.path.exists(config.save_model_path):
if os.path.exists(config.save_model_path):
    print("최고 성능 모델 로드 중...")
    inference_model = AutoModelForSequenceClassification.from_pretrained(f"{config.save_model_path}_{TIMESTAMP}")
    inference_tokenizer = AutoTokenizer.from_pretrained(f"{config.save_model_path}_{TIMESTAMP}")
    inference_model = inference_model.to(device)
    inference_model.eval()
    print("✅ 최고 성능 모델 로드 완료")
else:
    print("⚠️  저장된 모델이 없습니다. 현재 모델 사용")
    inference_model = model
    inference_tokenizer = tokenizer


테스트 데이터 추론 시작...
테스트 데이터: 59,928개
최고 성능 모델 로드 중...
✅ 최고 성능 모델 로드 완료


In [28]:
# 테스트 데이터셋 및 데이터로더 생성
test_dataset = ReviewDataset(
    test_df["review"],
    None,  # 테스트 데이터는 라벨 없음
    inference_tokenizer,
    config.max_length,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=config.eval_batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✅ 테스트 데이터셋 준비 완료")


✅ 테스트 데이터셋 준비 완료


In [29]:
# 추론 실행
print("추론 실행 중...")
inference_model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Inference"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        outputs = inference_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=-1)
        predictions.extend(batch_predictions.cpu().numpy())

print(f"✅ 추론 완료: {len(predictions):,}개 예측")


추론 실행 중...


Inference:   0%|          | 0/118 [00:00<?, ?it/s]

✅ 추론 완료: 59,928개 예측


In [30]:
# 제출 파일 생성
print("제출 파일 생성 중...")

# 예측 결과를 데이터프레임에 추가
test_df["pred"] = predictions

# 클래스별 예측 분포 확인
unique_predictions, counts = np.unique(predictions, return_counts=True)
print("\n클래스별 예측 분포:")
for pred, count in zip(unique_predictions, counts):
    percentage = (count / len(predictions)) * 100
    class_name = LABEL_MAPPING.get(pred, f"클래스 {pred}")
    print(f"   {class_name} ({pred}): {count:,}개 ({percentage:.1f}%)")

# 샘플 제출 파일 로드
sample_submission = pd.read_csv("data/sample_submission.csv")

# ID를 기준으로 병합하여 제출 파일 생성
submission_df = sample_submission[["ID"]].merge(
    test_df[["ID", "pred"]], 
    left_on="ID", 
    right_on="ID", 
    how="left"
)

# 제출 파일 검증
assert len(submission_df) == len(sample_submission), f"길이 불일치: {len(submission_df)} vs {len(sample_submission)}"
assert submission_df["pred"].isin([0, 1, 2, 3]).all(), "모든 예측값은 [0, 1, 2, 3] 범위에 있어야 합니다"
assert not submission_df["pred"].isnull().any(), "예측값에 null 값이 있으면 안됩니다"
assert not submission_df["ID"].isnull().any(), "ID 컬럼에 null 값이 있으면 안됩니다"

print("✅ 제출 파일 검증 통과")

# 제출 파일 저장
submission_path = f"{config.save_model_path}_output_{TIMESTAMP}.csv"
submission_df.to_csv(submission_path, index=False)
print(f"✅ 제출 파일 저장 완료: {submission_path}")

# WandB 종료
if config.use_wandb:
    wandb.finish()
    print("✅ WandB 세션 종료")

print("\n🎉 모든 작업 완료!")


제출 파일 생성 중...

클래스별 예측 분포:
   강한 부정 (0): 24,895개 (41.5%)
   약한 부정 (1): 5,377개 (9.0%)
   약한 긍정 (2): 20,879개 (34.8%)
   강한 긍정 (3): 8,777개 (14.6%)
✅ 제출 파일 검증 통과
✅ 제출 파일 저장 완료: ./DAPT_fine/newly_gen_added_output_2025-10-28_01-51-51.csv


epoch,▁▃▅▆█
learning_rate,█▆▅▃▁
train_accuracy,▁▆▇▇█
train_f1,▁▆▇▇█
train_loss,█▃▂▂▁
val_accuracy,▁▅▇██
val_f1,▁▅▇██
val_loss,█▂▁▂▃
epoch,5
learning_rate,0
train_accuracy,0.92124


✅ WandB 세션 종료

🎉 모든 작업 완료!
